In [58]:
import os, glob, sys
import matplotlib.pyplot as plt
import h5py
%matplotlib inline
from skimage.io import imread, imshow, imsave
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import time
from eval import *
from ShowColors import *
from ImportUtil import *
%env CITYSCAPES_DATASET = /home/rvygon/data/

%load_ext autoreload
%autoreload 2

env: CITYSCAPES_DATASET=/home/rvygon/data/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
#run this cell once
#%run  cityscapesscripts/preparation/createTrainIdLabelImgs
def upd_print(str):
            sys.stdout.write('\r')       
            sys.stdout.write(str)
            sys.stdout.flush()

In [3]:
x_val, y_val,files = importBatch(10,0,0,'val',scale = 4)
from keras.utils import to_categorical

y_val = to_categorical(y_val)

[[0.00000e+00 4.77296e+05]
 [1.00000e+00 4.53860e+04]
 [2.00000e+00 2.61651e+05]
 [4.00000e+00 2.86800e+03]
 [5.00000e+00 1.68800e+04]
 [6.00000e+00 2.91100e+03]
 [7.00000e+00 4.95100e+03]
 [8.00000e+00 1.46077e+05]
 [9.00000e+00 2.42000e+02]
 [1.00000e+01 4.27310e+04]
 [1.10000e+01 4.03400e+03]
 [1.20000e+01 1.80800e+03]
 [1.30000e+01 1.23560e+05]
 [1.40000e+01 1.27100e+03]
 [1.50000e+01 4.87600e+03]
 [1.60000e+01 1.41200e+03]
 [1.70000e+01 1.24000e+03]
 [1.80000e+01 2.58500e+03]
 [1.90000e+01 1.68941e+05]]


(10, 256, 512, 20)

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def preprocess_pic(X, y):    
    y=to_categorical(y)
    return X, y

def keras_generator(batch_size,scale):
    while True:
        batch_nums = np.random.randint(low=1000, high=2000, size=batch_size)
        batch_input = []
        batch_output = []
        
        for index in batch_nums:
            input_labels, input_images = getData(1, index,type='train')
            input_image = input_images[0]
            input_label = input_labels[0]           
            
            X_img = imread(input_image)
            if (scale != 0):
                X_new = np.zeros((int(X_img.shape[0] / scale), int(X_img.shape[1] / scale),3))
                k = 0
                for x in X_img[::scale]:
                    X_new[k]=x[::scale]
                    k+=1
                X_img = X_new
            y_img = imread(input_label)
            if (scale != 0):
                y_new = np.zeros((int(y_img.shape[0] / scale), int(y_img.shape[1] / scale)))
                k = 0
                for y in y_img[::scale]:
                    y_new[k] = y[::scale]
                    k += 1
                y_img = y_new
            
            X_img, y_img = preprocess_pic(X_img, y_img)
            
            batch_input.append(X_img)
            batch_output.append(y_img)
            
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)        
        yield batch_x, batch_y
        

In [5]:
#UNET https://github.com/zhixuhao/unet/blob/master/model.py
def unet(pretrained_weights = None,input_size = (256,512,3)):
    inputs = Input(input_size)
  
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)    
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))#drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(20, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [6]:
model = unet()
generator = keras_generator(1,4)
model_checkpoint = ModelCheckpoint('unet_RV.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(generator,steps_per_epoch=1500,epochs=8,callbacks=[model_checkpoint], validation_data=(x_val,y_val),validation_steps=20)

/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/8
  73/1500 [>.............................] - ETA: 9:51 - loss: 4.1426 - acc: 0.2622

KeyboardInterrupt: 

In [68]:
SCALE = 4
#tic = time.time()
#x_val, y_val, filenames = importBatch(500,0,0,'val',SCALE)
#toc = time.time()
#print('Load validation batch:', toc - tic)
def eval_model(model):          
        tic = time.time()
        x_pred = model.predict(x_val,verbose=1)
        toc = time.time()
        print('Predict:', toc - tic)
        tic = time.time()
        new_x=np.argmax(x_pred,axis=3)
        
        """ UPSCALING
        new_new_x = np.zeros((new_x.shape[0],new_x.shape[1]*SCALE,new_x.shape[2]*SCALE))
        
        for i in range(x_pred.shape[0]):
            new_new_x[i]=UpscaleImg(new_x[i],SCALE,0)
            upd_print(("Upscaled %d images" % i))
        print()
        new_new_x=new_new_x.astype(int)"""
        new_x=new_x.astype(int)
        
        
        toc = time.time()
        print('Reshape and upscale:', toc - tic)
        tic = time.time()
        """   SAVE
        cityscapesPath = os.environ['CITYSCAPES_DATASET']        
        for i in range(len(filenames)):
            impath = os.path.join(cityscapesPath,'results', filenames[i].split('/')[7]+'.png')           
            imsave(impath, new_new_x[i])
            upd_print("Saved %d images" % i)            
        print()
        toc = time.time()
        print('Save files:', toc -tic) """
        tic = time.time()
        score = eval_preds(new_x,y_val)
        toc = time.time()
        print('eval:', toc-tic)
        return score
tic = time.time()
model = load_model('unet_RV.hdf5')
toc = time.time()
print('Load model:', toc-tic)
print(eval_model(model))

Load model: 10.215620517730713
500/500 [==============================] - 41s 81ms/step
Predict: 40.52906632423401
Reshape and upscale: 2.7739717960357666
processed 499 imagesroad : 0.8246603522441442
sidewalk : 0.3333296681731544
building : 0.5812949191123348
wall : 0.0
fence : 0.0
pole : 0.0
traffic light : 0.0
traffic sign : 0.0
vegetation : 0.6616570463523643
terrain : 0.06623498510073546
sky : 0.6146912729570579
person : 0.003754189915612346
rider : 0.0
car : 0.4089478411413947
truck : 0.0
bus : 0.0
train : 0.0
motorcycle : 0.0
bicycle : 0.0
eval: 5.4663896560668945
0.20838302810431147
